In [1]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime

# ================================
# Google Sheets Authentication
# ================================
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name(
    "python-459011-73acbdff4227.json", scope
)
client = gspread.authorize(creds)

# ================================
# Load Master Sheet
# ================================
master_sheet = client.open_by_key(
    "1fGDYnO2M-PWqMQTBDO78qbgLC-EZKiJIly68_2D-TEA"
).worksheet("Master2")

index = get_as_dataframe(master_sheet, evaluate_formulas=True)
index.dropna(how="all", inplace=True)
index.reset_index(drop=True, inplace=True)

# ================================
# Normalize Column Names (CRITICAL FIX)
# ================================
index.columns = (
    index.columns
    .astype(str)
    .str.strip()
    .str.replace("\n", " ")
)

print("📌 Master Sheet Columns:", index.columns.tolist())

# ================================
# Validate Active Flag Column
# ================================
if "Active Flag" not in index.columns:
    raise ValueError(
        f"'Active Flag' column not found. "
        f"Available columns: {index.columns.tolist()}"
    )

# ================================
# Filter Active Projects
# ================================
index["Active Flag"] = pd.to_numeric(index["Active Flag"], errors="coerce")
active_projects = index[index["Active Flag"] == 1]

print(f"✅ Active Projects Found: {len(active_projects)}")

# ================================
# SQL Server Connection
# ================================
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

# ================================
# Start Processing Projects
# ================================
for idx, row in active_projects.iterrows():

    project_name = row["Project Name"]
    schema = str(row["SQL Schema Name"]).strip("[]")
    sheet_key = row["Google Sheet Key"]

    print(f"\n🚀 Processing Project: {project_name} | Schema: {schema}")

    # ----------------------------
    # Open Google Sheet
    # ----------------------------
    try:
        sheet = client.open_by_key(sheet_key)
        AppLog_Sheet = sheet.worksheet("Supervisor Log")
        Coverage_Sheet = sheet.worksheet("Coverage")
        Attendence_Sheet = sheet.worksheet("Attendence")
        Daywise_Sheet = sheet.worksheet("Day-Wise Summary")
    except Exception as e:
        print(f"❌ Sheet error for {project_name}: {e}")
        continue

    # ----------------------------
    # SQL Processing
    # ----------------------------
    try:
        with pyodbc.connect(conn_str) as conn:

            # =========================
            # Supervisor App Log
            # =========================
            app_query = f"""
            SELECT
                a.SupervisorAppLogId,
                a.ManagerId AS [Supervisor ID],
                a.ManagerName AS [Supervisor Name],
                b.UserName,
                a.UserFullName,
                a.UserDesignation,
                a.LISStoreCode,
                a.StoreName,
                a.ModuleName,
                a.Count,
                a.Time,
                CAST(a.Time AS DATE) AS LogDate
            FROM [{database}].[{schema}].[SupervisorAppLogReport] a
            LEFT JOIN [{database}].[{schema}].[UserMaster] b
                ON a.UserId = b.UserId
            WHERE MONTH(a.Time) = MONTH(GETDATE() - 1)
              AND YEAR(a.Time) = YEAR(GETDATE() - 1)
            """

            AppLog = pd.read_sql(app_query, conn)
            AppLog_Sheet.batch_clear(["A:M"])
            set_with_dataframe(AppLog_Sheet, AppLog)
            print("✅ AppLog Uploaded")

            # =========================
            # Coverage
            # =========================
            coverage_query = f"""
            SELECT *,
                   TRY_CONVERT(DATE, VisitDate, 105) AS DateOnly
            FROM [{database}].[{schema}].[ISPStoreAttendance_NormalExport]
            WHERE MONTH(PARSE(VisitDate AS datetime USING 'en-GB')) = MONTH(GETDATE() - 1)
              AND [Year] = YEAR(GETDATE() - 1)
              AND (UserName LIKE '%SUP%' OR UserName LIKE '%MCR-TL%')
            """

            Coverage = pd.read_sql(coverage_query, conn)
            Coverage_Sheet.batch_clear(["A:X"])
            set_with_dataframe(Coverage_Sheet, Coverage)
            print("✅ Coverage Uploaded")

            # =========================
            # Attendance
            # =========================
            attendance_query = f"""
            SELECT *
            FROM (
                SELECT *,
                       ROW_NUMBER() OVER (
                           PARTITION BY UserName, FullName, [Date]
                           ORDER BY [In-Time] DESC
                       ) AS RN,
                       CASE
                           WHEN UserName LIKE '%SUP%' OR UserName LIKE '%MCR-TL%' THEN 1
                           ELSE 0
                       END AS SupFlag
                FROM [{database}].[{schema}].[ISPAttendanceMasterExport]
                WHERE [Month] = MONTH(GETDATE() - 1)
                  AND [Year] = YEAR(GETDATE() - 1)
                  AND UserName NOT LIKE '%test%'
            ) t
            WHERE RN = 1
            """

            Attendence = pd.read_sql(attendance_query, conn)
            Attendence.drop(columns=["RN"], inplace=True)

            Attendence.at[0, "LastUpdated"] = datetime.now().strftime(
                "%Y-%m-%d %H:%M:%S"
            )

            Attendence_Sheet.batch_clear(["A:U"])
            set_with_dataframe(Attendence_Sheet, Attendence)
            print("✅ Attendance Uploaded")

            # =========================
            # Day-Wise Summary
            # =========================
            sup_df = Attendence[Attendence["SupFlag"] == 1].copy()
            sup_df["Date"] = pd.to_datetime(
                sup_df["Date"], dayfirst=True, errors="coerce"
            ).dt.strftime("%d-%m-%Y")

            summary = sup_df[["UserName", "FullName", "Date"]].drop_duplicates()

            Coverage["DateOnly"] = pd.to_datetime(
                Coverage["DateOnly"], errors="coerce"
            ).dt.strftime("%d-%m-%Y")

            Coverage["TimeDuration"] = (
                Coverage["TimeDuration"]
                .astype(str)
                .str.replace(r"[^0-9:]", "00:00", regex=True)
            )

            Coverage["TimeDuration"] = pd.to_timedelta(
                Coverage["TimeDuration"], errors="coerce"
            ).fillna(pd.Timedelta(0))

            agg = Coverage.groupby(["UserName", "DateOnly"]).agg(
                First_Login=("CheckIn-Time", "min"),
                Last_Logout=("CheckOut-Time", "max"),
                CFT=("TimeDuration", "sum"),
                Outlets=("LISStoreCode", "nunique")
            ).reset_index()

            agg["CFT"] = agg["CFT"].apply(
                lambda x: f"{int(x.total_seconds()//3600):02d}:{int((x.total_seconds()%3600)//60):02d}"
            )

            agg.rename(columns={"DateOnly": "Date"}, inplace=True)
            summary = summary.merge(agg, on=["UserName", "Date"], how="left").fillna("-")

            Daywise_Sheet.batch_clear(["A:H"])
            set_with_dataframe(Daywise_Sheet, summary)
            print("✅ Day-Wise Summary Uploaded")

            # =========================
            # Update Last Run Time
            # =========================
            index.at[idx, "Last Run Time"] = datetime.now().strftime(
                "%Y-%m-%d %H:%M:%S"
            )

    except Exception as e:
        print(f"❌ Error in project {project_name}: {e}")

# ================================
# Update Master Sheet
# ================================
master_sheet.clear()
set_with_dataframe(master_sheet, index)
print("\n🎯 Master Sheet Updated Successfully")


FileNotFoundError: [Errno 2] No such file or directory: 'python-459011-73acbdff4227.json'